In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark Intro") \
    .getOrCreate()

## Map and Reduce

We've talked about these concepts in the intro_python module, but we didn't talk explicity about reductions.

MAP

```python
def squarer(a):
    return a**2
```

REDUCE

The prototypical reduce function is the sum function. A reducer takes two values: the first value is called an "accumulator" and the second value is the next value in the list. The goal of the reduce function is to transform the iterable into a new object that is "accumulated" as one iterates through the items in the iterable:

```python
def summer(a,b):
    return a + b
```

In [ ]:
import pandas as pd

orders = pd.read_csv('data/orders.csv').to_dict(orient='records')

In [11]:
# We will start with the basic reduce function in Python
# NOTE: we have previously been using for comprehensions in python for mapping, 
# but there is also a built-in "map" function
from functools import reduce

In [ ]:
def make_total_sale(order):
    return order['quantity_ordered'] * order['price_each']

# What does this return???
map(make_total_sale, orders)

In [ ]:
def summer(a,b):
    return a + b

# Get the total sales!
reduce(summer, map(make_total_sale, orders))

In [ ]:
# Note, reduce takes an optional "initial value":

# See what happens when you change it from 0!
reduce(summer, map(make_total_sale, orders), 0.0)

In [ ]:
# Challenge: make a "map" and a "reduce" that gets the max number of items ordered (in one line item)!

# Your code here

In [ ]:
# Sometimes our data comes in JSON and it might be nested. Take a look at this format: 

import json

with open('data/orders.json') as f:
    orders = [json.loads(l) for l in f]

orders

In [ ]:
# Challenge: Find the sum of the total number of sales with this new format.
# Everything should be accomplished in a map and then a reduce:

# Your code here

In [ ]:
# Spark lets us read JSON files and create dataframes with nested items!

orders = spark.read.json('data/orders.json')

orders.printSchema()

In [ ]:
# All "DataFrames" in Spark are backed by an RDD. 
# We can access the RDD to perform simple operations, just like in python,
# on the underlying data:

# Spark performs all operations lazily. "collect" tells it to gather all the items
# in a list and send it back to the Driver node. 
# If you try and collect too much data, your memory will blow up!

orders.rdd.filter(lambda r: r.customer.country == 'Belgium').collect()

In [ ]:
# Challenge

# Let's try and repeat the previous operation, of finding the total sales, 
# but now with the spark RDD. 
# You will need to read the Spark Documentation and find the following functions: map, reduce
# which are available as methods directly on the RDD!
# play around: 


# your code

In [ ]:
# Spark thinks about where the Data lives. 
# RDD's have a concept of "key, value"
# This is implemented simply as a Tuple: (k,v)

# If we create an RDD that takes that form, a tuple of (k,v)
# we can use operations like "reduceByKey"!


# Challenge

# Get the total sales by country, by first mapping your RDD into a tuple (k,v) where
# the Key is the country, then reducing by summing the total sales. 

## Parquet

Parquet is another data format that plays well with Spark. 
 
It's a "flat file" format, like JSON or CSV, but it contains extra information about types, allows for "predicate pushdown", is column-oriented, and has first-class support for nested columns!

Predicate pushdown means that Spark doesn't have to read all the data from the disk! It can avoid certain sections of disk altogether because Parquet knows that we don't want that data.

In [42]:
orders = spark.read.parquet('data/orders')

In [ ]:
# Here, Spark won't read any information about countries other than Belgium!
# NOTE: the nested type!

orders.createOrReplaceTempView('orders')

res = spark.sql("""
SELECT count(order_number)
FROM orders 
WHERE customer.country = 'Belgium'
""".strip())

res.show()

## Nested types in SQL!

How do we deal with these pesky nested types now? 

Spark SQL gives us the Map/Reduce formulas to deal with nested "Array" types!

map = TRANSFORM
reduce = AGGREGATE

You can look at the documentation to see exactly how they work. 

In [ ]:
# Bonus Challenge: 

# Try to reproduce what we did before, getting the total sales, in Spark SQL, using TRANSFORM and AGGREGATE on the individuals "line_items" and then summing over the rows to get the total sales